This is the project that takes a number of files (images of newspapers). Input a search word and it will search through each image. If there is such a word in the image, it will detect faces, crop them and paste onto a contact sheet and show the contact sheet as outcome. Otherwise, it will return "No faces found". The project uses OCR for text detection, OpenCV (haar cascade) for face detection, and PIL for crop and paste images. The project was submitted as part of University of Michigan's Python 3 Programming Specialisation in July 2020. 

In [ ]:
#import packages
import zipfile
from PIL import Image
from PIL import ImageDraw
import os
import cv2 as cv

In [ ]:
#unzip, name the folder as 'unzip_folder'
with zipfile.ZipFile('readonly/images.zip', 'r') as my_zip:
    #print names of files
    print(my_zip.namelist()) 
    my_zip.extractall('unzip_folder')

In [ ]:
#give the directory
directory = 'unzip_folder/'
folder = os.listdir(directory)

In [ ]:
#detect_faces function
#detect_faces in each image
def detect_faces(file):
    face_cascade = cv.CascadeClassifier('readonly/haarcascade_frontalface_default.xml')
    path = os.path.join(directory, file)
    img = cv.imread(path)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray)
    faces_list = faces.tolist()
    if len(faces_list) != 0:
        print("There are {} faces detected in {}".format(len(faces_list), file))
        #crop the faces and paste onto a contact sheet
        img = Image.open(path)
        width = int(img.width/2)
        height = int(img.height/3)
        contact_sheet = Image.new(img.mode, (width,height))
        x=0
        y=0
        i =0
        for face in faces_list:
            img=Image.open(path)
            left, upper, right, lower = face
            im_crop = img.crop((left, upper, left+right, upper+lower))
            contact_sheet.paste(im_crop, (x, y))
            if (x+right) < (contact_sheet.width - 100):
                x = x + right
                y = y
            else:
                x = 0
                y = y+320
        display(contact_sheet)
    elif len(faces_list) == 0:
        print("But there were no faces in that file")
        print("-------------------------------------")
    return

In [ ]:
#find the search word in images in the folder
def find_the_word(search):
    for file in folder:
        path = os.path.join(directory, file)
        img = Image.open(path)
        img = img.convert("L")
        img.save("greyscale.jpg")
        text = pytesseract.image_to_string(Image.open('greyscale.jpg'))
        if search in text:
            print("Results found in file {}".format(file))
            detect_faces(file)
    return

In [ ]:
#input search word
search = input("Please enter your search word ")
find_the_word(search)